In [14]:
%config IPCompleter.greedy=True

### Tensors

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x=torch.tensor([[1,1],[-1,-1]])
print(x)
y=torch.tensor([[0,0],[1,1]])
print(y)
print(torch.add(x,y))
print(y.add_(x))
x0 = x.view(4)
print(x0)
x=torch.empty(5,3)
z = torch.randn(4,4,4)
z0 = z.view(-1,8,2)
print(z0)
print(z0.size())
print(torch.cuda.is_available())

tensor([[ 1,  1],
        [-1, -1]])
tensor([[0, 0],
        [1, 1]])
tensor([[1, 1],
        [0, 0]])
tensor([[1, 1],
        [0, 0]])
tensor([ 1,  1, -1, -1])
tensor([[[ 1.0313e+00, -1.3674e+00],
         [-1.8269e-01,  2.4855e-01],
         [-9.2092e-01,  8.7597e-01],
         [ 2.0277e+00, -4.0857e-01],
         [-5.6988e-01, -6.2629e-01],
         [-6.2347e-01, -1.1789e+00],
         [-6.7962e-01, -1.9917e+00],
         [ 2.4747e-01,  2.6083e-01]],

        [[ 1.4116e+00, -4.0260e-01],
         [-1.6202e+00,  1.3105e-01],
         [-7.6170e-02, -1.4254e+00],
         [ 8.2033e-02, -2.0899e+00],
         [-1.0674e+00, -4.5187e-01],
         [-2.4708e-01,  2.2166e+00],
         [ 1.3980e+00,  5.9151e-01],
         [ 2.1210e-01,  1.1238e+00]],

        [[-1.1211e+00,  2.2370e-01],
         [ 2.4903e-01,  7.3688e-01],
         [ 2.0341e-01,  3.1554e-01],
         [-1.0985e+00,  2.8133e-01],
         [-9.5189e-01, -9.3773e-01],
         [ 1.6298e+00,  1.0430e+00],
         [ 7.7593e-01

###  AutoGrad

In [3]:
x = torch.rand(2,2,requires_grad=True)
#print(x)
y = x + 2
print(y.grad_fn)
z = y * y * 3
z_mean = z.mean()
print(z)
print(z_mean)
z_mean.backward()
print(x.grad)

tensor([[24.4583, 13.1158],
        [18.8220, 18.7894]], grad_fn=<MulBackward0>)
tensor(18.7964, grad_fn=<MeanBackward1>)
tensor([[4.2830, 3.1364],
        [3.7572, 3.7539]])


In [43]:
x = torch.randn(3,3,requires_grad=True)
print(x)

tensor([[-0.7367, -1.2929,  0.1704],
        [-0.4890, -0.2523,  0.0469],
        [-0.2459,  0.2969, -2.6363]], requires_grad=True)


### MNIST TASK

In [4]:
from torch.autograd import Variable
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(784,15)
        self.fc2 = nn.Linear(15,10)
        self.fc3 = nn.Linear(10,10)
        
    def forward(self,x):
        x=torch.sigmoid(self.fc1(x))
        x=torch.sigmoid(self.fc2(x))
        x=torch.sigmoid(self.fc3(x))
        return x
    
    
    def train(self, data_loader, epochs=10, bs=64, criterion=nn.CrossEntropyLoss()):
        opt = optim.SGD(self.parameters(), lr=0.01)
        for epoch in range(epochs):
            print("Epoch number=" + str(epoch))
            for xb,yb in data_loader:
                xb = xb.reshape(-1,28*28)
                pred = self(xb)
                print("Accuracy of model =" + str(accuracy(pred,yb)))
                #print(pred)
                #print(yb)
                loss = criterion(pred, yb)
                #print("loss=", loss)
                ###
                loss.backward()
                opt.step()
                opt.zero_grad() 
                
            
            
            


In [5]:
from pathlib import Path
import requests

# Extracting the data
DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
        
# Creating tensors instead of numpy arrays
x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid))
print(y_train)
print(y_valid)
print(y_valid.size())
# Creating Dataset
tr_data_set = TensorDataset(x_train, y_train)
tr_data_loader = DataLoader(tr_data_set, batch_size=64)

tensor([5, 0, 4,  ..., 8, 4, 8])
tensor([3, 8, 6,  ..., 5, 6, 8])
torch.Size([10000])


In [6]:
from __future__ import print_function
from torch.autograd import Variable
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(784,15)
        self.fc2 = nn.Linear(15,10)
        self.fc3 = nn.Linear(10,10)
        
    def forward(self,x):
        x=torch.sigmoid(self.fc1(x))
        x=torch.sigmoid(self.fc2(x))
        x=torch.sigmoid(self.fc3(x))
        return x
    

In [7]:
from torchvision.datasets import MNIST
from torchvision import transforms
_tasks = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
## Load MNIST Dataset and apply transformations
mnist = MNIST("data", download=True, train=True, transform=_tasks)


In [27]:
from torch.utils.data.sampler import SubsetRandomSampler

split = int(0.8 * len(mnist))
index_list = list(range(len(mnist)))
train_idx, valid_idx = index_list[:split], index_list[split:]
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)
trainloader = DataLoader(mnist, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(mnist, batch_size=256, sampler=val_sampler)
print(trainloader)
for data, target in trainloader:
    for x in range(1):
        print(data.size())
        print(target)
    break
mnist

torch.Size([256, 1, 28, 28])
tensor([5, 7, 0, 5, 1, 0, 6, 1, 4, 0, 5, 7, 5, 5, 1, 0, 8, 4, 4, 1, 3, 8, 8, 6,
        8, 1, 2, 7, 3, 8, 4, 4, 2, 4, 1, 4, 9, 9, 5, 1, 8, 2, 9, 3, 5, 5, 3, 1,
        3, 6, 4, 4, 7, 8, 5, 4, 3, 8, 1, 7, 0, 1, 8, 3, 5, 3, 1, 8, 9, 7, 1, 4,
        8, 0, 4, 1, 0, 6, 2, 0, 3, 7, 4, 8, 9, 3, 4, 1, 8, 5, 6, 7, 2, 2, 4, 9,
        1, 5, 4, 0, 1, 5, 1, 1, 4, 7, 5, 3, 9, 3, 4, 7, 2, 8, 4, 1, 0, 7, 7, 0,
        3, 1, 5, 4, 7, 6, 8, 5, 3, 8, 2, 0, 4, 3, 5, 5, 0, 1, 9, 2, 1, 7, 1, 3,
        0, 5, 2, 0, 6, 2, 1, 2, 5, 9, 1, 9, 9, 2, 5, 3, 4, 0, 1, 1, 2, 7, 5, 6,
        1, 6, 2, 6, 9, 5, 8, 6, 0, 4, 6, 0, 9, 4, 3, 6, 6, 0, 6, 3, 9, 7, 0, 9,
        0, 8, 5, 5, 2, 8, 4, 5, 2, 3, 9, 8, 5, 2, 5, 6, 7, 5, 3, 8, 7, 8, 9, 6,
        9, 3, 0, 6, 1, 9, 4, 9, 5, 1, 0, 7, 2, 8, 8, 7, 6, 7, 7, 9, 8, 9, 7, 6,
        1, 3, 6, 5, 6, 7, 1, 9, 4, 6, 1, 1, 1, 3, 2, 7])


Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None

In [11]:
model = Net()
from torch import optim
from IPython.core.debugger import set_trace

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(1, 11): ## run the model for 10 epochs
    train_loss, valid_loss = [], []

    ## training part 
    model.train()
    for data, target in trainloader:
        optimizer.zero_grad()

        ## 1. forward propagation
        output = model(data)
        
        ## 2. loss calculation
        loss = loss_function(output, target)
        
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer.step()
        
        train_loss.append(loss.item())
        
    ## evaluation part 
    model.eval()
    for data, target in validloader:
        output = model(data)
        loss = loss_function(output, target)
        valid_loss.append(loss.item())

    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))



RuntimeError: size mismatch, m1: [7168 x 28], m2: [784 x 15] at /build/python-pytorch/src/pytorch-1.0.1-cuda/aten/src/TH/generic/THTensorMath.cpp:940

In [ ]:
    model.train()
    optimizer.zero_grad()

    y_ = model(X)
    loss = criterion(y_, y)

    loss.backward()
    optimizer.step()
